In [ ]:
%%bash
rm -r sample_data

# **Extract all the GSM links based in the give GSE accession IDs**

In [ ]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
from concurrent.futures import ThreadPoolExecutor, as_completed

def extract_all_GSM_links(GSE_url):
    response = requests.get(GSE_url)
    content = response.content
    soup = BeautifulSoup(content, "html.parser")
    tr_elems = soup.find_all('tr')

    GSM_links = []
    for tr_elem in tr_elems:
        if tr_elem.get('valign') == 'top' and 'GSM' in tr_elem.text:
            links = tr_elem.find_all('a')
            for link in links:
                href = link.get('href')
                if href and href.startswith("/geo/query/acc.cgi?acc="):
                    https = f"https://www.ncbi.nlm.nih.gov{href}"
                    GSM_links.append(https)
    return GSM_links

def process_row(row):
    accession = row['accession_found']
    GSE_url = f'https://www.ncbi.nlm.nih.gov/geo/query/acc.cgi?acc={accession}'
    return extract_all_GSM_links(GSE_url)

# Read the DataFrame
df = pd.read_csv("/content/srr_dates.csv")

# Prepare a dictionary to hold our results
results = {}

# Use ThreadPoolExecutor to parallelize the extraction
with ThreadPoolExecutor(max_workers=12) as executor:
    # Submit tasks to the executor
    future_to_row = {executor.submit(process_row, row): index for index, row in df.iterrows()}

    # Collect the results as they are completed
    for future in as_completed(future_to_row):
        index = future_to_row[future]
        try:
            results[index] = future.result()
        except Exception as exc:
            print(f'Generated an exception: {exc}')
        else:
            print(f'Row {index} done')

    # Update the DataFrame with the results
for index, GSM_links in results.items():
    # Convert the list of links to a string representation if you want to store it in a single cell
    # You can choose a separator that suits your needs; here, I use a comma.
    links_str = ','.join(GSM_links)
    df.at[index, 'GSM_links'] = links_str
df

Row 5 done
Row 8 done
Row 2 done
Row 1 done
Row 10 done
Row 0 done
Row 11 done
Row 4 done
Row 3 done
Row 9 done
Row 14 done
Row 7 done
Row 6 done
Row 12 done
Row 13 done
Row 17 done
Row 16 done
Row 15 done
Row 18 done
Row 21 done
Row 19 done
Row 23 done
Row 25 done
Row 26 done
Row 24 done
Row 22 done
Row 28 done
Row 27 done
Row 31 done
Row 30 done
Row 29 done
Row 32 done
Row 20 done
Row 33 done
Row 37 done
Row 38 done
Row 34 done
Row 36 done
Row 39 done
Row 35 done
Row 40 done
Row 41 done
Row 44 done
Row 42 done
Row 43 done
Row 45 done
Row 47 done
Row 50 done
Row 46 done
Row 51 done
Row 49 done
Row 48 done
Row 52 done
Row 54 done
Row 53 done
Row 56 done
Row 55 done
Row 57 done
Row 58 done
Row 59 done
Row 60 done
Row 62 done
Row 64 done
Row 63 done
Row 61 done
Row 65 done
Row 66 done
Row 67 done
Row 71 done
Row 70 done
Row 69 done
Row 68 done
Row 72 done
Row 74 done
Row 75 done
Row 77 done
Row 76 done
Row 79 done
Row 78 done
Row 73 done
Row 81 done
Row 80 done
Row 82 done
Row 83 done
Ro

,preprint_doi,published_doi_x,accession_found,GSM_links
0,10.1101/001107,10.1101/gr.181016.114,GSE58871,https://www.ncbi.nlm.nih.gov/geo/query/acc.cgi...
1,10.1101/002006,10.1186/1471-2164-15-962,GSE54119,https://www.ncbi.nlm.nih.gov/geo/query/acc.cgi...
2,10.1101/004853,10.1371/journal.pgen.1004402,GSE50759,https://www.ncbi.nlm.nih.gov/geo/query/acc.cgi...
3,10.1101/006171,10.1371/journal.pgen.1004663,GSE57483,https://www.ncbi.nlm.nih.gov/geo/query/acc.cgi...
4,10.1101/006270,10.7554/eLife.03915,GSE58423,https://www.ncbi.nlm.nih.gov/geo/query/acc.cgi...
...,...,...,...,...
1392,10.1101/2020.12.28.424586,10.1172/jci.insight.147280,GSE163649,https://www.ncbi.nlm.nih.gov/geo/query/acc.cgi...
1393,10.1101/2020.12.29.424680,10.3389/fnins.2021.671249,GSE160992,https://www.ncbi.nlm.nih.gov/geo/query/acc.cgi...
1394,10.1101/2020.12.30.424365,10.7554/eLife.70514,GSE163775,https://www.ncbi.nlm.nih.gov/geo/query/acc.cgi...
1395,10.1101/2020.12.30.424776,10.15252/msb.202110207,GSE158319,https://www.ncbi.nlm.nih.gov/geo/query/acc.cgi...


In [ ]:
# Save the modified DataFrame back to the CSV file
df.to_csv("/content/srr_dates_final.csv", index=False)

For every GSM links separated by "," and within a cell, let's create a new row for every GSM link

In [ ]:
import pandas as pd
df=pd.read_csv('/content/srr_dates_final.csv')
df['GSM_links'] = df['GSM_links'].astype(str)
df['GSM_links'] = df['GSM_links'].str.replace(' ', '')
df['GSM_links'] = df['GSM_links'].str.split(',')
df = df.explode('GSM_links')
df

In [ ]:
df.to_csv('/content/srr_dates_exploded.csv')

# **Extract all the SRX links based in the give GSM links**

In [ ]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
from concurrent.futures import ThreadPoolExecutor, as_completed

def extract_all_SRX_links(GSM_url):
    # Send an HTTP GET request to the URL
    response = requests.get(GSM_url)

    # Check if the request was successful (status code 200)
    if response.status_code == 200:
        # Get the content of the response
        content = response.content

        # Create a BeautifulSoup object and specify the parser
        soup = BeautifulSoup(content, "html.parser")

        # Find all the <strong> elements in the HTML
        strong_elems = soup.find_all('strong')

        # Variable to track if 'Relations' was found
        relations_found = False

        # Iterate over the <strong> elements
        for strong_elem in strong_elems:
            # Check if 'Relations' is present in the text of the <strong> element
            if 'Relations' in strong_elem.text:
                relations_found = True
                break
        href_found = []
        # If 'Relations' was found, proceed with extracting the links
        if relations_found:
            tr_elems = soup.find_all('tr')
            for tr_elem in tr_elems:
                # Check if the <tr> element has 'valign' attribute set to 'top' and contains 'SRA' in its text
                if tr_elem.get('valign') == 'top' and 'SRA' in tr_elem.text:
                    links = tr_elem.find_all('a')
                    href_found = None
                    for link in links:
                        href_found = link.get('href')
                        if 'SRX' in href_found:
                            return href_found
        # If 'Relations' was not found
        else:
            return "SRX not found"

    else:
        return "Request failed. Status code:", response.status_code

def process_row(row):
    GSM_url = row['GSM_links']
    return extract_all_SRX_links(GSM_url)

df1 = pd.read_csv("/content/srr_dates_exploded.csv")

# Prepare a dictionary to hold our results
results = {}

# Use ThreadPoolExecutor to parallelize the extraction
with ThreadPoolExecutor(max_workers=12) as executor:
    # Submit tasks to the executor
    future_to_row = {executor.submit(process_row, row): index for index, row in df1.iterrows()}

    # Collect the results as they are completed
    for future in as_completed(future_to_row):
        index = future_to_row[future]
        try:
            results[index] = future.result()
        except Exception as exc:
            print(f'Generated an exception: {exc}')
        else:
            print(f'Row {index} done')

    # Update the DataFrame with the results
for index, href_found in results.items():
    # Convert the list of links to a string representation if you want to store it in a single cell
    # You can choose a separator that suits your needs; here, I use a comma.
    df1.at[index, 'SRX_links'] = href_found
df1

流式输出内容被截断，只能显示最后 5000 行内容。
Row 60179 done
Row 60178 done
Row 60180 done
Row 60181 done
Row 60182 done
Row 60183 done
Row 60184 done
Row 60166 done
Row 60177 done
Row 60172 done
Row 60186 done
Row 60187 done
Row 60190 done
Row 60189 done
Row 60185 done
Row 60188 done
Row 60191 done
Row 60192 done
Row 60193 done
Row 60195 done
Row 60194 done
Row 60196 done
Row 60197 done
Row 60200 done
Row 60199 done
Row 60202 done
Row 60203 done
Row 60201 done
Row 60198 done
Row 60207 done
Row 60204 done
Row 60205 done
Row 60208 done
Row 60206 done
Row 60209 done
Row 60210 done
Row 60211 done
Row 60212 done
Row 60219 done
Row 60220 done
Row 60214 done
Row 60218 done
Row 60213 done
Row 60221 done
Row 60222 done
Row 60223 done
Row 60217 done
Row 60215 done
Row 60216 done
Row 60224 done
Row 60225 done
Row 60226 done
Row 60227 done
Row 60228 done
Row 60229 done
Row 60233 done
Row 60232 done
Row 60234 done
Row 60235 done
Row 60230 done
Row 60231 done
Row 60236 done
Row 60237 done
Row 60238 done
Row 60240 don

,preprint_doi,published_doi_x,accession_found,GSM_links,SRX_links
0,10.1101/001107,10.1101/gr.181016.114,GSE58871,https://www.ncbi.nlm.nih.gov/geo/query/acc.cgi...,https://www.ncbi.nlm.nih.gov/sra?term=SRX631821
1,10.1101/001107,10.1101/gr.181016.114,GSE58871,https://www.ncbi.nlm.nih.gov/geo/query/acc.cgi...,https://www.ncbi.nlm.nih.gov/sra?term=SRX631822
2,10.1101/001107,10.1101/gr.181016.114,GSE58871,https://www.ncbi.nlm.nih.gov/geo/query/acc.cgi...,https://www.ncbi.nlm.nih.gov/sra?term=SRX631823
3,10.1101/001107,10.1101/gr.181016.114,GSE58871,https://www.ncbi.nlm.nih.gov/geo/query/acc.cgi...,https://www.ncbi.nlm.nih.gov/sra?term=SRX631824
4,10.1101/001107,10.1101/gr.181016.114,GSE58871,https://www.ncbi.nlm.nih.gov/geo/query/acc.cgi...,https://www.ncbi.nlm.nih.gov/sra?term=SRX631825
...,...,...,...,...,...
65170,10.1101/2020.12.30.424817,10.1186/s13059-021-02453-5,GSE163896,https://www.ncbi.nlm.nih.gov/geo/query/acc.cgi...,https://www.ncbi.nlm.nih.gov/sra?term=SRX9738885
65171,10.1101/2020.12.30.424817,10.1186/s13059-021-02453-5,GSE163896,https://www.ncbi.nlm.nih.gov/geo/query/acc.cgi...,https://www.ncbi.nlm.nih.gov/sra?term=SRX9738886
65172,10.1101/2020.12.30.424817,10.1186/s13059-021-02453-5,GSE163896,https://www.ncbi.nlm.nih.gov/geo/query/acc.cgi...,https://www.ncbi.nlm.nih.gov/sra?term=SRX9738887
65173,10.1101/2020.12.30.424817,10.1186/s13059-021-02453-5,GSE163896,https://www.ncbi.nlm.nih.gov/geo/query/acc.cgi...,https://www.ncbi.nlm.nih.gov/sra?term=SRX9738888


In [ ]:
df1.to_csv("/content/srr_dates_exploded_returned.csv")

At the very begining, let's fill the nan values in the column "SRX_links" with "SRX not found". In the next step, let's take the whole "srr_dates_exploded_returned.csv" into two parts: the column "SRX_links", there are two categories: one is the with the link-"https://www.ncbi.nlm.nih.gov/sra?term=SRX"; the other is the "SRX not found".

In [ ]:
import pandas as pd
df_3 = pd.read_csv('/content/srr_dates_exploded_returned.csv')
df_3['SRX_links'] = df_3['SRX_links'].fillna('SRX not found')

df_3_new = df_3.loc[df_3['SRX_links']=='SRX not found']
df_3_new.to_csv('/content/SRR_dates_equals_to_GSE_published_dates_new.csv')

In [ ]:
df_2 = df_3.loc[~(df_3['SRX_links']=='SRX not found')]
df_2.to_csv('/content/srr_dates_need_to_be_extracted.csv')

# **Extract all the SRR dates**

In [ ]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
from concurrent.futures import ThreadPoolExecutor, as_completed

def extract_SRR_dates(SRX_url):
    response = requests.get(SRX_url)
    if response.status_code == 200:
        content = response.content
        soup = BeautifulSoup(content, "html.parser")
        count = 0
        th_elems = soup.find_all('th')
        for th_elem in th_elems:
            if 'Published' in th_elem.text:
                td_elems = soup.find_all('td')
                for i, td_elem in enumerate(td_elems, start=1):
                    if td_elem.get('align') == 'right':
                        count += 1
                        if count == 3:
                           SRR_published_dates = td_elems[i].text
                return SRR_published_dates
        return "Published section not found"
    else:
        return "Failed to fetch the page"

def process_row(row):
    SRX_url = row['SRX_links']
    return extract_SRR_dates(SRX_url)

df2 = pd.read_csv("/content/srr_dates_need_to_be_extracted.csv")

# Prepare a dictionary to hold our results
results = {}

# Use ThreadPoolExecutor to parallelize the extraction
with ThreadPoolExecutor(max_workers=12) as executor:
    # Submit tasks to the executor
    future_to_row = {executor.submit(process_row, row): index for index, row in df2.iterrows()}

    # Collect the results as they are completed
    for future in as_completed(future_to_row):
        index = future_to_row[future]
        try:
            results[index] = future.result()
        except Exception as exc:
            print(f'Generated an exception: {exc}')
        else:
            print(f'Row {index} done')

    # Update the DataFrame with the results
for index, SRR_published_dates in results.items():
    # Convert the list of links to a string representation if you want to store it in a single cell
    # You can choose a separator that suits your needs; here, I use a comma.
    df2.at[index, 'SRR_dates'] = SRR_published_dates
df2

流式输出内容被截断，只能显示最后 5000 行内容。
Row 49759 done
Row 49760 done
Row 49761 done
Row 49762 done
Row 49765 done
Row 49764 done
Row 49766 done
Row 49763 done
Row 49768 done
Row 49769 done
Row 49767 done
Row 49770 done
Row 49771 done
Row 49774 done
Row 49772 done
Row 49776 done
Row 49773 done
Row 49778 done
Row 49777 done
Row 49775 done
Row 49780 done
Row 49781 done
Row 49784 done
Row 49779 done
Row 49783 done
Row 49782 done
Row 49789 done
Row 49788 done
Row 49786 done
Row 49787 done
Row 49790 done
Row 49785 done
Row 49791 done
Row 49794 done
Row 49792 done
Row 49796 done
Row 49793 done
Row 49795 done
Row 49797 done
Row 49798 done
Row 49800 done
Row 49799 done
Row 49802 done
Row 49801 done
Row 49803 done
Row 49805 done
Row 49806 done
Row 49807 done
Row 49808 done
Row 49804 done
Row 49809 done
Row 49810 done
Row 49811 done
Row 49813 done
Row 49812 done
Row 49814 done
Row 49815 done
Row 49820 done
Row 49819 done
Row 49816 done
Row 49817 done
Row 49818 done
Row 49821 done
Row 49823 done
Row 49822 don

,preprint_doi,published_doi_x,accession_found,GSM_links,SRX_links,SRR_dates
0,10.1101/001107,10.1101/gr.181016.114,GSE58871,https://www.ncbi.nlm.nih.gov/geo/query/acc.cgi...,https://www.ncbi.nlm.nih.gov/sra?term=SRX631821,2014-09-23
1,10.1101/001107,10.1101/gr.181016.114,GSE58871,https://www.ncbi.nlm.nih.gov/geo/query/acc.cgi...,https://www.ncbi.nlm.nih.gov/sra?term=SRX631822,2014-09-23
2,10.1101/001107,10.1101/gr.181016.114,GSE58871,https://www.ncbi.nlm.nih.gov/geo/query/acc.cgi...,https://www.ncbi.nlm.nih.gov/sra?term=SRX631823,2014-09-23
3,10.1101/001107,10.1101/gr.181016.114,GSE58871,https://www.ncbi.nlm.nih.gov/geo/query/acc.cgi...,https://www.ncbi.nlm.nih.gov/sra?term=SRX631824,2014-09-23
4,10.1101/001107,10.1101/gr.181016.114,GSE58871,https://www.ncbi.nlm.nih.gov/geo/query/acc.cgi...,https://www.ncbi.nlm.nih.gov/sra?term=SRX631825,2014-09-23
...,...,...,...,...,...,...
54754,10.1101/2020.12.30.424817,10.1186/s13059-021-02453-5,GSE163896,https://www.ncbi.nlm.nih.gov/geo/query/acc.cgi...,https://www.ncbi.nlm.nih.gov/sra?term=SRX9738885,2021-07-17
54755,10.1101/2020.12.30.424817,10.1186/s13059-021-02453-5,GSE163896,https://www.ncbi.nlm.nih.gov/geo/query/acc.cgi...,https://www.ncbi.nlm.nih.gov/sra?term=SRX9738886,2021-07-17
54756,10.1101/2020.12.30.424817,10.1186/s13059-021-02453-5,GSE163896,https://www.ncbi.nlm.nih.gov/geo/query/acc.cgi...,https://www.ncbi.nlm.nih.gov/sra?term=SRX9738887,2021-07-17
54757,10.1101/2020.12.30.424817,10.1186/s13059-021-02453-5,GSE163896,https://www.ncbi.nlm.nih.gov/geo/query/acc.cgi...,https://www.ncbi.nlm.nih.gov/sra?term=SRX9738888,2021-07-17


In [ ]:
df2.to_csv('/content/extracted_SRR_dates.csv')

# **Extract GSE ID public date**

## Extract GSE ID public date for code-based found GSE IDs

## The following GSE IDs do not associate with sequencing data

In [ ]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
from concurrent.futures import ThreadPoolExecutor, as_completed

def extract_GSE_dates(GSE_url):
    # Send an HTTP GET request to the URL
    response = requests.get(GSE_url)
    # Check if the request was successful (status code 200)
    if response.status_code == 200:
        # Get the content of the response
        content = response.content
        # Create a BeautifulSoup object and specify the parser
        soup = BeautifulSoup(content, "html.parser")
        tr_elems = soup.find_all('tr')
        published_dates = []
        for tr_elem in tr_elems:
            if tr_elem.get('valign') == 'top' and 'Status' in tr_elem.text:
                td_elems = tr_elem.find_all('td')  # Find td elements within this tr element
                date_text = td_elems[1].text.strip()  # Get the text from the second td element
                GSE_dates = date_text

        published_dates = GSE_dates[10:22]
        return published_dates

def process_row(row):
    accession = row['accession_found']
    GSE_url = f'https://www.ncbi.nlm.nih.gov/geo/query/acc.cgi?acc={accession}'
    return extract_GSE_dates(GSE_url)

# Read the DataFrame
df2 = pd.read_csv("/content/SRR_dates_equals_to_GSE_published_dates_new.csv")

# Prepare a dictionary to hold our results
results = {}

# Use ThreadPoolExecutor to parallelize the extraction
with ThreadPoolExecutor(max_workers=12) as executor:
    # Submit tasks to the executor
    future_to_row = {executor.submit(process_row, row): index for index, row in df2.iterrows()}

    # Collect the results as they are completed
    for future in as_completed(future_to_row):
        index = future_to_row[future]
        try:
            results[index] = future.result()
        except Exception as exc:
            print(f'Generated an exception: {exc}')
        else:
            print(f'Row {index} done')

    # Update the DataFrame with the results
for index, published_dates in results.items():
    df2.at[index, 'GSE_dates'] = published_dates
df2

FileNotFoundError: [Errno 2] No such file or directory: '/content/SRR_dates_equals_to_GSE_published_dates_new.csv'

In [ ]:
df2.to_csv("/content/extracted_gse_dates.csv")

## The following GSE IDs associate with sequencing data(stored in accession ID begin with SRR)

In [ ]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
from concurrent.futures import ThreadPoolExecutor, as_completed

def extract_GSE_dates(GSE_url):
    # Send an HTTP GET request to the URL
    response = requests.get(GSE_url)
    # Check if the request was successful (status code 200)
    if response.status_code == 200:
        # Get the content of the response
        content = response.content
        # Create a BeautifulSoup object and specify the parser
        soup = BeautifulSoup(content, "html.parser")
        tr_elems = soup.find_all('tr')
        published_dates = []
        for tr_elem in tr_elems:
            if tr_elem.get('valign') == 'top' and 'Status' in tr_elem.text:
                td_elems = tr_elem.find_all('td')  # Find td elements within this tr element
                date_text = td_elems[1].text.strip()  # Get the text from the second td element
                GSE_dates = date_text

        published_dates = GSE_dates[10:22]
        return published_dates

def process_row(row):
    accession = row['accession_found']
    GSE_url = f'https://www.ncbi.nlm.nih.gov/geo/query/acc.cgi?acc={accession}'
    return extract_GSE_dates(GSE_url)

# Read the DataFrame
df3 = pd.read_csv("/content/extracted_SRR_dates_final.csv")

# Prepare a dictionary to hold our results
results = {}

# Use ThreadPoolExecutor to parallelize the extraction
with ThreadPoolExecutor(max_workers=12) as executor:
    # Submit tasks to the executor
    future_to_row = {executor.submit(process_row, row): index for index, row in df3.iterrows()}

    # Collect the results as they are completed
    for future in as_completed(future_to_row):
        index = future_to_row[future]
        try:
            results[index] = future.result()
        except Exception as exc:
            print(f'Generated an exception: {exc}')
        else:
            print(f'Row {index} done')

    # Update the DataFrame with the results
for index, published_dates in results.items():
    df3.at[index, 'GSE_dates'] = published_dates
df3

Row 3 done
Row 2 done
Row 4 done
Row 9 done
Row 10 done
Row 7 done
Row 0 done
Row 5 done
Row 8 done
Row 6 done
Row 1 done
Row 11 done
Row 12 done
Row 13 done
Row 14 done
Row 22 done
Row 15 done
Row 21 done
Row 19 done
Row 16 done
Row 25 done
Row 23 done
Row 18 done
Row 20 done
Row 24 done
Row 17 done
Row 27 done
Row 26 done
Row 28 done
Row 30 done
Row 31 done
Row 32 done
Row 29 done
Row 36 done
Row 35 done
Row 33 done
Row 37 done
Row 39 done
Row 40 done
Row 34 done
Row 41 done
Row 42 done
Row 43 done
Row 44 done
Row 47 done
Row 48 done
Row 46 done
Row 38 done
Row 45 done
Row 52 done
Row 54 done
Row 51 done
Row 50 done
Row 49 done
Row 59 done
Row 56 done
Row 55 done
Row 57 done
Row 63 done
Row 61 done
Row 65 done
Row 64 done
Row 62 done
Row 66 done
Row 60 done
Row 67 done
Row 68 done
Row 69 done
Row 71 done
Row 72 done
Row 73 done
Row 53 done
Row 70 done
Row 75 done
Row 79 done
Row 58 done
Row 76 done
Row 77 done
Row 80 done
Row 74 done
Row 78 done
Row 81 done
Row 83 done
Row 86 done
Ro

,preprint_doi,published_doi_x,accession_found,GSM_links,SRX_links,SRR_dates,GSE_dates
0,10.1101/001107,10.1101/gr.181016.114,GSE58871,https://www.ncbi.nlm.nih.gov/geo/query/acc.cgi...,https://www.ncbi.nlm.nih.gov/sra?term=SRX631821,9/23/2014,"Sep 23, 2014"
1,10.1101/006171,10.1371/journal.pgen.1004663,GSE57483,https://www.ncbi.nlm.nih.gov/geo/query/acc.cgi...,https://www.ncbi.nlm.nih.gov/sra?term=SRX539569,7/22/2015,"May 09, 2014"
2,10.1101/006270,10.7554/eLife.03915,GSE58423,https://www.ncbi.nlm.nih.gov/geo/query/acc.cgi...,https://www.ncbi.nlm.nih.gov/sra?term=SRX591752,"7/22/2015,6/15/2014,7/20/2014","Jun 13, 2014"
3,10.1101/008136,10.1371/journal.pgen.1005118,GSE57608,https://www.ncbi.nlm.nih.gov/geo/query/acc.cgi...,https://www.ncbi.nlm.nih.gov/sra?term=SRX541132,7/22/2015,"Apr 17, 2015"
4,10.1101/012757,10.1534/g3.115.019497,GSE63488,https://www.ncbi.nlm.nih.gov/geo/query/acc.cgi...,https://www.ncbi.nlm.nih.gov/sra?term=SRX763987,12/17/2014,"Dec 17, 2014"
...,...,...,...,...,...,...,...
1229,10.1101/2020.12.28.424586,10.1172/jci.insight.147280,GSE163649,https://www.ncbi.nlm.nih.gov/geo/query/acc.cgi...,https://www.ncbi.nlm.nih.gov/sra?term=SRX9711745,12/25/2020,"Dec 22, 2020"
1230,10.1101/2020.12.29.424680,10.3389/fnins.2021.671249,GSE160992,https://www.ncbi.nlm.nih.gov/geo/query/acc.cgi...,https://www.ncbi.nlm.nih.gov/sra?term=SRX9450936,5/13/2021,"May 11, 2021"
1231,10.1101/2020.12.30.424365,10.7554/eLife.70514,GSE163775,https://www.ncbi.nlm.nih.gov/geo/query/acc.cgi...,https://www.ncbi.nlm.nih.gov/sra?term=SRX9724367,10/9/2021,"Oct 07, 2021"
1232,10.1101/2020.12.30.424776,10.15252/msb.202110207,GSE158319,https://www.ncbi.nlm.nih.gov/geo/query/acc.cgi...,https://www.ncbi.nlm.nih.gov/sra?term=SRX9164576,11/1/2020,"Oct 31, 2020"


In [ ]:
df3.to_csv("/content/SRR_and_GSE_dates.csv")

## Extract GSE ID public date for manual check based GSE IDs

### Concatenate all GSE IDs into a large file, including those associating with as well as not associating with sequencing data

In [ ]:
import pandas as pd
df_in_both_do_not_have_sequencing_data = pd.read_csv("do_not_have_sequencing_data_in_both.csv",index_col=None)
df_only_in_pub_do_not_have_sequencing_data = pd.read_csv("do_not_have_sequencing_data_only_in_pub.csv",index_col=None)

df_in_both_do_not_have_sequencing_data.rename(columns={'primary_GSE_in_both': 'accession_found'}, inplace=True)
df_only_in_pub_do_not_have_sequencing_data.rename(columns={'primary_GSE_only_in_pub': 'accession_found'}, inplace=True)

df_in_both_do_not_have_sequencing_data['Sharing Scenario']= 'GSE IDs in both'
df_only_in_pub_do_not_have_sequencing_data['Sharing Scenario']= 'GSE IDs only in pub'

df_do_not_have = pd.concat([df_in_both_do_not_have_sequencing_data, df_only_in_pub_do_not_have_sequencing_data])
df_do_not_have['Method to obtain data'] = 'Manual check'
df_do_not_have['Associating Sequence data'] = 'None'
df_do_not_have.drop(columns=['Unnamed: 0'], inplace=True)
df_do_not_have

,preprint_doi,publication_doi,accession_found,bioproject_ID,Sharing Scenario,Method to obtain data,Associating Sequence data
0,10.1101/2021.10.27.466206,10.1016/j.envres.2022.112890,GSE183071,PRJNA761110,GSE IDs in both,Manual check,None
1,10.1101/465716,10.1016/j.gpb.2018.11.003,GSE121231,PRJNA761110,GSE IDs in both,Manual check,None
2,10.1101/815068,10.1016/j.neo.2020.08.002,GSE138737,PRJNA576925,GSE IDs in both,Manual check,None
3,10.1101/391763,10.1038/s41419-018-1096-6,GSE114345,PRJNA471056,GSE IDs in both,Manual check,None
4,10.1101/546952,10.1038/s41467-020-15188-x,GSE121947,PRJNA499112,GSE IDs in both,Manual check,None
...,...,...,...,...,...,...,...
29,10.1101/503409,10.7554/eLife.44642,GSE124201,PRJNA510984,GSE IDs only in pub,Manual check,None
30,10.1101/2020.06.22.165225,10.1371/journal.pbio.3000849,GSE154770,PRJNA647399,GSE IDs only in pub,Manual check,None
31,10.1101/2020.06.22.165225,10.1371/journal.pbio.3000849,GSE154768,PRJNA647400,GSE IDs only in pub,Manual check,None
32,10.1101/2020.06.22.165225,10.1371/journal.pbio.3000849,GSE154769,PRJNA647401,GSE IDs only in pub,Manual check,None


### Deal with the 990 GSE IDs that associate with sequencing data

In [ ]:
import pandas as pd
df_990_have_sequencing_data = pd.read_csv("final_990_GSE_IDs_from_manual_check_with_metadata.csv",index_col=None)
df_990_have_sequencing_data.drop(columns=['Unnamed: 0','Unnamed: 0.1','Unnamed: 0.2','SRR_published_dates','Library_Strategy','Library_Source','Scientific_Name','statistics'], inplace=True)
df_990_have_sequencing_data['Method to obtain data'] = 'Manual check'
df_990_have_sequencing_data['Associating Sequence data'] = 'Yes'

In [ ]:
df_990_have_sequencing_data

,preprint_doi,publication_doi,accession_found,bioproject_ID,Sharing Scenario,Method to obtain data,Associating Sequence data
0,10.1101/459289,10.1007/s13238-019-0650-z,GSE55600,PRJNA240203,GSE IDs in both,Manual check,Yes
1,10.1101/425348,10.1016/j.celrep.2019.03.108,GSE115929,PRJNA476539,GSE IDs in both,Manual check,Yes
2,10.1101/393876,10.1016/j.celrep.2019.07.084,GSE133660,PRJNA552100,GSE IDs in both,Manual check,Yes
3,10.1101/624957,10.1016/j.celrep.2019.08.089,GSE125523,PRJNA516680,GSE IDs in both,Manual check,Yes
4,10.1101/600932,10.1016/j.celrep.2019.09.017,GSE120520,PRJNA493256,GSE IDs in both,Manual check,Yes
...,...,...,...,...,...,...,...
985,10.1101/2020.03.08.982769,10.7554/eLife.56523,GSE144919,PRJNA605337,GSE IDs only in pub,Manual check,Yes
986,10.1101/2020.03.08.982769,10.7554/eLife.56523,GSE144920,PRJNA605338,GSE IDs only in pub,Manual check,Yes
987,10.1101/2021.05.06.442900,10.7554/eLife.69843,GSE186004,PRJNA771684,GSE IDs only in pub,Manual check,Yes
988,10.1101/2021.06.15.448594,10.3390/cancers13174250,GSE153595,PRJNA643285,GSE IDs only in pub,Manual check,Yes


### Concatenate

In [ ]:
df_need_to_obtain_GSE_public_date_from_manual = pd.concat([df_990_have_sequencing_data,df_do_not_have])
df_need_to_obtain_GSE_public_date_from_manual.to_csv("whole_df_need_to_obtain_GSE_public_date_from_manual.csv")

In [ ]:
print(len(df_need_to_obtain_GSE_public_date_from_manual))

1120


### Extract

In [ ]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
from concurrent.futures import ThreadPoolExecutor, as_completed

def extract_GSE_dates(GSE_url):
    # Send an HTTP GET request to the URL
    response = requests.get(GSE_url)
    # Check if the request was successful (status code 200)
    if response.status_code == 200:
        # Get the content of the response
        content = response.content
        # Create a BeautifulSoup object and specify the parser
        soup = BeautifulSoup(content, "html.parser")
        tr_elems = soup.find_all('tr')
        published_dates = []
        for tr_elem in tr_elems:
            if tr_elem.get('valign') == 'top' and 'Status' in tr_elem.text:
                td_elems = tr_elem.find_all('td')  # Find td elements within this tr element
                date_text = td_elems[1].text.strip()  # Get the text from the second td element
                GSE_dates = date_text

        published_dates = GSE_dates[10:22]
        return published_dates

def process_row(row):
    accession = row['accession_found']
    GSE_url = f'https://www.ncbi.nlm.nih.gov/geo/query/acc.cgi?acc={accession}'
    return extract_GSE_dates(GSE_url)

# Read the DataFrame
df_whole = pd.read_csv("whole_df_need_to_obtain_GSE_public_date_from_manual.csv")

# Prepare a dictionary to hold our results
results = {}

# Use ThreadPoolExecutor to parallelize the extraction
with ThreadPoolExecutor(max_workers=12) as executor:
    # Submit tasks to the executor
    future_to_row = {executor.submit(process_row, row): index for index, row in df_whole.iterrows()}

    # Collect the results as they are completed
    for future in as_completed(future_to_row):
        index = future_to_row[future]
        try:
            results[index] = future.result()
        except Exception as exc:
            print(f'Generated an exception: {exc}')
        else:
            print(f'Row {index} done')

    # Update the DataFrame with the results
for index, published_dates in results.items():
    df_whole.at[index, 'GSE_dates'] = published_dates
df_whole

Row 0 done
Row 6 done
Row 2 done
Row 9 done
Row 4 done
Row 11 done
Row 1 done
Row 3 done
Row 7 done
Row 12 done
Row 8 done
Row 5 done
Row 14 done
Row 13 done
Row 10 done
Row 15 done
Row 18 done
Row 20 done
Row 17 done
Row 22 done
Row 19 done
Row 16 done
Row 26 done
Row 21 done
Row 24 done
Row 25 done
Row 27 done
Row 23 done
Row 31 done
Row 30 done
Row 28 done
Row 29 done
Row 35 done
Row 32 done
Row 36 done
Row 37 done
Row 38 done
Row 39 done
Row 43 done
Row 44 done
Row 41 done
Row 40 done
Row 42 done
Row 46 done
Row 45 done
Row 48 done
Row 47 done
Row 33 done
Row 49 done
Row 34 done
Row 50 done
Row 51 done
Row 53 done
Row 55 done
Row 56 done
Row 58 done
Row 61 done
Row 52 done
Row 62 done
Row 54 done
Row 63 done
Row 57 done
Row 60 done
Row 59 done
Row 64 done
Row 65 done
Row 67 done
Row 66 done
Row 68 done
Row 73 done
Row 71 done
Row 74 done
Row 76 done
Row 77 done
Row 72 done
Row 69 done
Row 75 done
Row 79 done
Row 78 done
Row 80 done
Row 81 done
Row 83 done
Row 84 done
Row 86 done
Ro

,Unnamed: 0,preprint_doi,publication_doi,accession_found,bioproject_ID,Sharing Scenario,Method to obtain data,Associating Sequence data,GSE_dates
0,0,10.1101/459289,10.1007/s13238-019-0650-z,GSE55600,PRJNA240203,GSE IDs in both,Manual check,Yes,"Feb 26, 2019"
1,1,10.1101/425348,10.1016/j.celrep.2019.03.108,GSE115929,PRJNA476539,GSE IDs in both,Manual check,Yes,"Apr 23, 2019"
2,2,10.1101/393876,10.1016/j.celrep.2019.07.084,GSE133660,PRJNA552100,GSE IDs in both,Manual check,Yes,"Jul 02, 2019"
3,3,10.1101/624957,10.1016/j.celrep.2019.08.089,GSE125523,PRJNA516680,GSE IDs in both,Manual check,Yes,"Sep 01, 2019"
4,4,10.1101/600932,10.1016/j.celrep.2019.09.017,GSE120520,PRJNA493256,GSE IDs in both,Manual check,Yes,"Oct 15, 2019"
...,...,...,...,...,...,...,...,...,...
1115,29,10.1101/503409,10.7554/eLife.44642,GSE124201,PRJNA510984,GSE IDs only in pub,Manual check,NaN,"Dec 03, 2019"
1116,30,10.1101/2020.06.22.165225,10.1371/journal.pbio.3000849,GSE154770,PRJNA647399,GSE IDs only in pub,Manual check,NaN,"Jul 21, 2020"
1117,31,10.1101/2020.06.22.165225,10.1371/journal.pbio.3000849,GSE154768,PRJNA647400,GSE IDs only in pub,Manual check,NaN,"Jul 21, 2020"
1118,32,10.1101/2020.06.22.165225,10.1371/journal.pbio.3000849,GSE154769,PRJNA647401,GSE IDs only in pub,Manual check,NaN,"Jul 21, 2020"


In [ ]:
df_whole.fillna('None', inplace=True)
FP = df_whole.loc[df_whole['GSE_dates']=='None']
FP

,Unnamed: 0,preprint_doi,publication_doi,accession_found,bioproject_ID,Sharing Scenario,Method to obtain data,Associating Sequence data,GSE_dates
1065,75,10.1101/403543,10.1101/gr.243378.118,None,PRJNA488283,GSE IDs in both,Manual check,None,None


In [ ]:
df_whole.drop(index=FP.index, inplace=True)

In [ ]:
print(len(df_whole)) # There are 1120 GSE IDs in total from manual check

1119


In [ ]:
from datetime import datetime

df_whole['GSE_dates'] = df_whole['GSE_dates'].apply(lambda x: datetime.strptime(x, '%b %d, %Y').strftime('%m/%d/%Y'))

df_whole.drop(columns=['Unnamed: 0'], inplace=True)

In [ ]:
df_whole.to_csv("whole_1120_df_with_GSE_public_date_from_manual_check.csv")